In [1]:
# Untuk pengolahan data
import pandas as pd
import numpy as np
from zipfile import ZipFile
from pathlib import Path

# Untuk visualisasi data
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
sns.set_palette('Set1')
sns.set()

# Untuk pemodelan
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.metrics import RootMeanSquaredError
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, Dense, Concatenate


# Untuk menghilangkan warnings saat plotting seaborn
import warnings
warnings.filterwarnings('ignore')

# Untuk mengupload file
import os

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")
rating = pd.read_excel(r'/content/gdrive/My Drive/Colab Notebooks/Dataset_Review_Restaurant.xlsx')

Mounted at /content/gdrive


In [3]:
from google.colab import drive
drive.mount("/content/gdrive")
place = pd.read_excel(r'/content/gdrive/My Drive/Colab Notebooks/Dataset_Restaurant.xlsx')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
merged_df = rating.merge(place, on='Place_ID')

In [5]:
merged_df

,User_ID,Place_ID,Ratings,Place,Image,URL,Rating,Num_Rating,Category,Address,Longitude,Latitude,Label_Kecamatan
0,1,12,1,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
1,4,12,2,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
2,7,12,2,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
3,22,12,2,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
4,22,12,3,Super Penyet Setiabudi,https://lh5.googleusercontent.com/p/AF1QipMufn...,https://www.google.com/maps/place/Super+Penyet...,4.4,5702,Ayam,Jl. Setia Budi No.207,110.411722,-7.058634,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
511319,17653,55,2,Logissteak Genuk indah,https://lh5.googleusercontent.com/p/AF1QipO3uB...,https://www.google.com/maps/place/Logissteak+G...,4.6,352,Steak,Jl. Kapas Utara Raya No.824,110.465362,-6.964662,5
511320,17669,55,3,Logissteak Genuk indah,https://lh5.googleusercontent.com/p/AF1QipO3uB...,https://www.google.com/maps/place/Logissteak+G...,4.6,352,Steak,Jl. Kapas Utara Raya No.824,110.465362,-6.964662,5
511321,17808,55,2,Logissteak Genuk indah,https://lh5.googleusercontent.com/p/AF1QipO3uB...,https://www.google.com/maps/place/Logissteak+G...,4.6,352,Steak,Jl. Kapas Utara Raya No.824,110.465362,-6.964662,5
511322,17906,55,3,Logissteak Genuk indah,https://lh5.googleusercontent.com/p/AF1QipO3uB...,https://www.google.com/maps/place/Logissteak+G...,4.6,352,Steak,Jl. Kapas Utara Raya No.824,110.465362,-6.964662,5


In [6]:
# Encode categorical variables
user_encoder = LabelEncoder()
place_encoder = LabelEncoder()

In [7]:
merged_df['user_encoded'] = user_encoder.fit_transform(merged_df['User_ID'])
merged_df['place_encoded'] = place_encoder.fit_transform(merged_df['Place_ID'])

In [8]:
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

In [9]:
# Define input layers
user_input = Input(shape=(1,))
place_input = Input(shape=(1,))

# Define embedding layers
user_embedding = Embedding(len(user_encoder.classes_), 10)(user_input)
place_embedding = Embedding(len(place_encoder.classes_), 10)(place_input)

# Flatten the embeddings
user_flatten = Flatten()(user_embedding)
place_flatten = Flatten()(place_embedding)

# Concatenate the embeddings
concatenated = Concatenate()([user_flatten, place_flatten])

# Define hidden layers
hidden1 = Dense(64, activation='relu')(concatenated)
hidden2 = Dense(32, activation='relu')(hidden1)

# Output layer
output = Dense(1)(hidden2)

In [10]:
model = Model(inputs=[user_input, place_input], outputs=output)

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

In [11]:
X_train = [train_df['user_encoded'], train_df['place_encoded']]
y_train = train_df['Ratings']

# Train the model
model.fit(X_train, y_train, epochs=3, batch_size=32)

Epoch 1/3
12784/12784 [==============================] - 75s 6ms/step - loss: 2.0402
Epoch 2/3
12784/12784 [==============================] - 68s 5ms/step - loss: 2.0003
Epoch 3/3
12784/12784 [==============================] - 77s 6ms/step - loss: 1.9707


In [12]:
# Prepare the test data
X_test = [test_df['user_encoded'], test_df['place_encoded']]
y_test = test_df['Ratings']

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print('Test Loss:', loss)

3196/3196 [==============================] - 4s 1ms/step - loss: 2.0319
Test Loss: 2.0319294929504395


In [15]:
# Get recommendations for a given user
user_id = '100'
user_index = user_encoder.transform([user_id])[0]

place_indices = np.arange(len(place_encoder.classes_))
user_indices = np.array([user_index] * len(place_encoder.classes_))

recommendations = model.predict([user_indices, place_indices])
recommendations = np.squeeze(recommendations)

# Get top 10 recommendations
top_indices = recommendations.argsort()[-10:][::-1]
top_places = place.iloc[top_indices]

# Display the recommendations
for _, row in top_places.iterrows():
    print('Place Name:', row['Place'])
    print('Address:', row['Address'])
    print('Ratings:', row['Rating'])
    print('Number of Ratings:', row['Num_Rating'])
    print('Category:', row['Category'])
    print('Image:', row['Image'])
    print('Url:', row['URL'])
    print('-----------------------------')

5/5 [==============================] - 0s 2ms/step
Place Name: My Kopi O!
Address: Jl. Sisingamangaraja No.75
Ratings: 4.4
Number of Ratings: 1748
Category: Restoran
Image: https://lh5.googleusercontent.com/p/AF1QipPODoVkKz8JPGwLXwtoR_BWOK_si8zKLKKWn-nC=w122-h92-k-no
Url: https://www.google.com/maps/place/My+Kopi+O!/data=!4m7!3m6!1s0x2e708b7897d809af:0xe3a9fbfac1abed96!8m2!3d-7.0153033!4d110.4193801!16s%2Fg%2F11b90cyvq2!19sChIJrwnYl3iLcC4Rlu2rwfr7qeM?authuser=0&hl=id&rclk=1
-----------------------------
Place Name: Ventura Taman Kuliner By Kekancan Group
Address: Jl. Ki Mangunsarkoro No.15
Ratings: 4.5
Number of Ratings: 6393
Category: Indonesia
Image: https://lh5.googleusercontent.com/p/AF1QipP6Y-Xem-E_Oi27oBV5TtICB7JGL0bAiT59Ywz5=w163-h92-k-no
Url: https://www.google.com/maps/place/Ventura+Taman+Kuliner+By+Kekancan+Group/data=!4m7!3m6!1s0x2e708b6827c0ef27:0xc248d067938c4f52!8m2!3d-6.9900131!4d110.4288788!16s%2Fg%2F1pzrdhnrd!19sChIJJ-_AJ2iLcC4RUk-Mk2fQSMI?authuser=0&hl=id&rclk=1
-----